# 0. 강화학습을 이용한 체스 (Reinforcement Learning Chess) - 2

※본 커널은 캐글을 통해 공유된 https://www.kaggle.com/arjanso/reinforcement-learning-chess-2-model-free-methods 의 내용을 기반으로 작성되었습니다.



# Reinforcement Learning Chess 
Reinforcement Learning Chess는 체스 AI를 개발하기 위해 강화 학습 알고리즘을 구현하는 일련의 Notebook입니다. 간단한 방법으로 다루어 질 수 있는 더 단순한 버전 (환경)으로 시작하여 본격적인 체스 인공 지능이 생길 때까지 이러한 개념을 점차적으로 확장합니다.

[**Notebook 1: Policy Iteration**](https://www.kaggle.com/arjanso/reinforcement-learning-chess-1-policy-iteration)  
[**Notebook 3: Q-networks**](https://www.kaggle.com/arjanso/reinforcement-learning-chess-3-q-networks)  
[**Notebook 4: Policy Gradients**](https://www.kaggle.com/arjanso/reinforcement-learning-chess-4-policy-gradients)  
[**Notebook 5: Monte Carlo Tree Search**](https://www.kaggle.com/arjanso/reinforcement-learning-chess-5-tree-search)  

# Notebook II: Model-free control
이 Notebook에서는 Notebook 1과 동일한 체스말 이동 environment를 사용합니다.이 Notebook에서는 정책평가(policy evaluation)는 후속 상태값(state values)과 전환확률(transition probabilities)을 해당 상태로 백업하여 상태값(state values)을 계산합니다. 문제는 이러한 확률은 일반적으로 실제 문제에서 알 수 없다는 것입니다. 운 좋게 이러한 알 수 없는 environment에서 작동 할 수 있는  기술이 있습니다. 이러한 기술은 environment에 대한 사전 지식을 활용하지 않는 model-free 기술입니다.

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import inspect

In [3]:
!pip install --upgrade git+https://github.com/arjangroen/RLC.git  # RLC is the Reinforcement Learning package

  Cloning https://github.com/arjangroen/RLC.git to /tmp/pip-req-build-p139r2vn
  Running command git clone -q https://github.com/arjangroen/RLC.git /tmp/pip-req-build-p139r2vn
  Created wheel for RLC: filename=RLC-0.3-cp36-none-any.whl size=22568 sha256=a479faa3517ebfb3ef3edc3450b927b7ece5a42c6abfa055583e108f8891d439
  Stored in directory: /tmp/pip-ephem-wheel-cache-7z00s2ye/wheels/04/68/a5/cb835cd3d76a49de696a942739c71a56bfe66d0d8ea7b4b446
Successfully built RLC


In [0]:
from RLC.move_chess.environment import Board
from RLC.move_chess.agent import Piece
from RLC.move_chess.learn import Reinforce

### 2.0.1 The environment
- 상태(state) 공간은 8x8의 그리드입니다
- 시작 상태(state) S는 왼쪽 위 사각형 (0, 0)을 기준으로 합니다.
- 종료 상태(state) F의 좌표는 (5, 7) 입니다.
- 한 상태(state)에서 다른 상태(state)로 이동할 때마다 보상(reward)을 1 감소시킵니다.
- 이 환경(environment)에 대한 최상의 정책(policy)은 가장 적은 수의 움직임으로 S로부터 F까지 움직이는 것입니다.

In [5]:
env = Board()
env.render()
env.visual_board

[['[S]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]'],
 ['[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]'],
 ['[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]'],
 ['[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]'],
 ['[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]'],
 ['[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]'],
 ['[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[ ]'],
 ['[ ]', '[ ]', '[ ]', '[ ]', '[ ]', '[F]', '[ ]', '[ ]']]

### 2.0.2 The agent
- 에이전트(agent)는 체스 말입니다. (킹, 퀸, 룩, 나이트, 비숍 등)
- 에이전트(agent)는 에이전트가 어떤 상태(state)에서 무엇을 하는지 결정하는 행동 정책(policy)을 갖고 있습니다.

In [0]:
p = Piece(piece='king')

### 2.0.3 Reinforce
- 강화(reinforce) 객체에는 체스 말을 움직이는 문제를 해결하기 위한 알고리즘이 포함되어 있습니다.
- 에이전트(agent)와 환경(environment)은 강화(reinforce) 객체의 속성(attributes)입니다.

In [0]:
r = Reinforce(p,env)

# 2.1 Monte Carlo Control

**Theory**  
* 우리는 환경(environment)을 알지 못하므로 현재 정책(policy)을 실행하여 에피소드를 처음부터 끝까지 샘플링합니다.
* 상태값(state value)이 아닌 행동값(action value)을 추정하려고합니다. 우리는 모델이 없는(model-free) 상황에서 작업하기 때문에 알려진 상태값(state value)만으로는 최선의 행동(action)을 선택하는 데 도움이 되지 않기 때문입니다.
* state-action value의 값은 해당 state-action의 첫 방문으로부터 이후의 return들로 정의됩니다.
* 이를 바탕으로 정책(policy)을 개선하고 알고리즘이 수렴 될 때까지 프로세스를 반복할 수 있습니다

![](http://incompleteideas.net/book/first/ebook/pseudotmp5.png)

**Implementation**

In [8]:
print(inspect.getsource(r.monte_carlo_learning))

    def monte_carlo_learning(self, epsilon=0.1):
        """
        Learn move chess through monte carlo control
        :param epsilon: exploration rate
        :return:
        """
        state = (0, 0)
        self.env.state = state

        # Play out an episode
        states, actions, rewards = self.play_episode(state, epsilon=epsilon)

        first_visits = []
        for idx, state in enumerate(states):
            action_index = actions[idx]
            if (state, action_index) in first_visits:
                continue
            r = np.sum(rewards[idx:])
            if (state, action_index) in self.agent.Returns.keys():
                self.agent.Returns[(state, action_index)].append(r)
            else:
                self.agent.Returns[(state, action_index)] = [r]
            self.agent.action_function[state[0], state[1], action_index] = \
                np.mean(self.agent.Returns[(state, action_index)])
            first_visits.append((state, action_index))
        #

**Demo**  
0.5의 높은 탐색 속도를 유지하면서 몬테 카를로 학습을 100 회 반복합니다

In [0]:
for k in range(100):
    eps = 0.5
    r.monte_carlo_learning(epsilon=eps)

In [10]:
r.visualize_policy()

[['↘', '↘', '→', '↓', '↘', '↖', '←', '↙'],
 ['↑', '↘', '→', '↖', '→', '↓', '↑', '↙'],
 ['↓', '↘', '→', '→', '↘', '↘', '↓', '↗'],
 ['↗', '↓', '↗', '↘', '↗', '↓', '↘', '↘'],
 ['←', '↘', '→', '↙', '↓', '↓', '↙', '↙'],
 ['↓', '↗', '↑', '↘', '→', '↙', '↓', '↑'],
 ['↘', '↓', '↗', '→', '↘', '↓', '↙', '←'],
 ['↗', '↙', '→', '↗', '→', 'F', '←', '↑']]


각 상태(state)에 대한 최상의 행동값(action value)

In [11]:
r.agent.action_function.max(axis=2).astype(int)

array([[-41, -40, -32, -51, -24, -20, -20, -56],
       [-37, -30, -37, -35, -23, -16, -32, -28],
       [-37, -24, -26, -19, -14, -13, -19, -51],
       [-30, -31, -23, -14, -18, -13, -10, -41],
       [-34, -50, -29, -22,  -7,  -9,  -8,  -7],
       [-39, -47, -24,  -8,  -4,  -4,  -5, -11],
       [-30, -12, -16,  -5,  -1,  -1,  -1,  -2],
       [-52, -11, -25,  -4,  -1,   0,  -1,  -4]])

# 2.2 Temporal Difference Learning 

**Theory**
* 정책(Policy) 반복과 마찬가지로 에피소드가 끝나기를 기다리지 않고 후속 상태행동(state-action)에서 상태행동값(state-action value)을 백업 할 수 있습니다.
* 후속 상태행동값(state-action value) 방향으로 상태행동값(state-action value)을 업데이트합니다.
* 알고리즘을 SARSA : State-Action-Reward-State-Action이라고합니다.
* Epsilon이 점차 낮아짐 (GLIE 속성)

**Implementation**

In [12]:
print(inspect.getsource(r.sarsa_td))

    def sarsa_td(self, n_episodes=1000, alpha=0.01, gamma=0.9):
        """
        Run the sarsa control algorithm (TD0), finding the optimal policy and action function
        :param n_episodes: int, amount of episodes to train
        :param alpha: learning rate
        :param gamma: discount factor of future rewards
        :return: finds the optimal policy for move chess
        """
        for k in range(n_episodes):
            state = (0, 0)
            self.env.state = state
            episode_end = False
            epsilon = max(1 / (1 + k), 0.05)
            while not episode_end:
                state = self.env.state
                action_index = self.agent.apply_policy(state, epsilon)
                action = self.agent.action_space[action_index]
                reward, episode_end = self.env.step(action)
                successor_state = self.env.state
                successor_action_index = self.agent.apply_policy(successor_state, epsilon)

                action_va

**Demonstration**

In [0]:
p = Piece(piece='king')
env = Board()
r = Reinforce(p,env)
r.sarsa_td(n_episodes=10000,alpha=0.2,gamma=0.9)

In [14]:
r.visualize_policy()

[['↓', '↘', '↙', '→', '↖', '→', '↘', '↖'],
 ['↘', '↓', '↘', '↘', '↑', '→', '↓', '↗'],
 ['↘', '↘', '↘', '↘', '↓', '↙', '↘', '↙'],
 ['←', '↘', '↘', '↓', '↘', '↙', '↑', '→'],
 ['↓', '↘', '↘', '↘', '↓', '↘', '↙', '↙'],
 ['↗', '↘', '↘', '↘', '↘', '↘', '↓', '↙'],
 ['↙', '↗', '↘', '↘', '↘', '↓', '↙', '↖'],
 ['←', '↙', '→', '→', '→', 'F', '←', '←']]


# 2.3 TD-lambda
**Theory**  
Monte Carlo에서는 full-depth 백업을 수행하고 Temporal Difference Learning에서는 1-step 백업을 수행합니다. 또한 백업 간격을 n 단계로 선택할 수도 있습니다. 그러나 n에 어떤 값을 선택해야합니까?

* TD lambda는 모든 n-step을 사용하고 factor lambda로 할인합니다
* 이것을 lambda-return이라고합니다
* TD-lambda는 eligibility-trace를 사용하여 이전에 발생한 상태를 추적합니다.
* 이런 식으로 행동가치(action-value)는 회고적으로 업데이트 될 수 있습니다

**Implementation**

In [15]:
print(inspect.getsource(r.sarsa_lambda))

    def sarsa_lambda(self, n_episodes=1000, alpha=0.05, gamma=0.9, lamb=0.8):
        """
        Run the sarsa control algorithm (TD lambda), finding the optimal policy and action function
        :param n_episodes: int, amount of episodes to train
        :param alpha: learning rate
        :param gamma: discount factor of future rewards
        :param lamb: lambda parameter describing the decay over n-step returns
        :return: finds the optimal move chess policy
        """
        for k in range(n_episodes):
            self.agent.E = np.zeros(shape=self.agent.action_function.shape)
            state = (0, 0)
            self.env.state = state
            episode_end = False
            epsilon = max(1 / (1 + k), 0.2)
            action_index = self.agent.apply_policy(state, epsilon)
            action = self.agent.action_space[action_index]
            while not episode_end:
                reward, episode_end = self.env.step(action)
                successor_state = self.env.

**Demonstration**

In [0]:
p = Piece(piece='king')
env = Board()
r = Reinforce(p,env)
r.sarsa_lambda(n_episodes=10000,alpha=0.2,gamma=0.9)

In [17]:
r.visualize_policy()

[['↓', '↘', '↘', '↙', '↘', '↙', '↘', '↙'],
 ['↘', '↓', '↓', '↓', '↑', '↙', '→', '↓'],
 ['↘', '↓', '↘', '↙', '↓', '←', '↓', '↙'],
 ['→', '↘', '↘', '↓', '↙', '↘', '↘', '→'],
 ['↘', '→', '↘', '↘', '↓', '↓', '↘', '↙'],
 ['↗', '↗', '↘', '↘', '↓', '↙', '↓', '↙'],
 ['→', '↘', '→', '↘', '↘', '↓', '↙', '↙'],
 ['→', '→', '↗', '→', '→', 'F', '←', '↖']]


# 2.4 Q-learning

**Theory**
* SARSA / TD0 에서는 행동값(action value)을 후속 행동값(action value)으로 백업합니다.
* SARSA-max / Q 학습 에서는 최대 행동값(action value)을 사용하여 백업합니다.

**Implementation**

In [18]:
print(inspect.getsource(r.sarsa_lambda))

    def sarsa_lambda(self, n_episodes=1000, alpha=0.05, gamma=0.9, lamb=0.8):
        """
        Run the sarsa control algorithm (TD lambda), finding the optimal policy and action function
        :param n_episodes: int, amount of episodes to train
        :param alpha: learning rate
        :param gamma: discount factor of future rewards
        :param lamb: lambda parameter describing the decay over n-step returns
        :return: finds the optimal move chess policy
        """
        for k in range(n_episodes):
            self.agent.E = np.zeros(shape=self.agent.action_function.shape)
            state = (0, 0)
            self.env.state = state
            episode_end = False
            epsilon = max(1 / (1 + k), 0.2)
            action_index = self.agent.apply_policy(state, epsilon)
            action = self.agent.action_space[action_index]
            while not episode_end:
                reward, episode_end = self.env.step(action)
                successor_state = self.env.

**Demonstration**

In [0]:
p = Piece(piece='king')
env = Board()
r = Reinforce(p,env)
r.q_learning(n_episodes=1000,alpha=0.2,gamma=0.9)

In [20]:
r.visualize_policy()

[['↘', '→', '↘', '↘', '↖', '→', '→', '↘'],
 ['↘', '↓', '↘', '↘', '↗', '↘', '←', '↗'],
 ['↘', '↘', '↘', '↘', '↘', '↓', '→', '←'],
 ['↘', '↘', '↓', '↘', '↘', '↓', '↓', '↑'],
 ['↗', '↙', '↘', '↘', '↘', '↙', '↘', '↑'],
 ['↘', '↓', '↘', '↘', '↘', '↘', '↓', '↙'],
 ['↓', '↗', '→', '↘', '↘', '↓', '↙', '↙'],
 ['↓', '←', '↙', '→', '→', 'F', '↙', '↖']]


In [21]:
r.agent.action_function.max(axis=2).round().astype(int)

array([[-5, -5, -5, -4, -4, -4, -4, -4],
       [-5, -5, -5, -4, -4, -4, -4, -3],
       [-4, -4, -4, -4, -4, -4, -3, -3],
       [-4, -3, -3, -3, -3, -3, -3, -3],
       [-3, -3, -3, -3, -3, -3, -3, -3],
       [-3, -3, -3, -2, -2, -2, -2, -2],
       [-3, -3, -2, -2, -1, -1, -1, -2],
       [-3, -3, -2, -2, -1,  0, -1, -1]])

# References
1. Reinforcement Learning: An Introduction  
   Richard S. Sutton and Andrew G. Barto  
   1st Edition  
   MIT Press, march 1998
2. RL Course by David Silver: Lecture playlist  
   https://www.youtube.com/watch?v=2pWv7GOvuf0&list=PLqYmG7hTraZDM-OYHWgPebj2MfCFzFObQ